In [ ]:
#|hide
from ntlights_processing_utils.core import *

# ~~Nightlights for Damage Assessment~~ Nightlights Processing Utilities
> ~~Using nightlights satellite imagery for typhoon damage assessment~~ Utilities for Processing Nightlights Data

## Install

`pip install git+https://github.com/butchland/ntlights-processing-utils.git`

## How to use

* ~~Download the nightlights satellite imagery for the period before and after the disaster~~
* ~~Download the geospatial admin boundaries for the areas concerned~~
* ~~Download and scrape the damage assessment situationers as ground truth~~
* ~~Preprocess the situationrers to extract the locations and damage assessment~~
* ~~Wrangle and link the damage assessment locations to the admin boundaries~~
* ~~Extract the radiance data from the nightlights satellite~~
* ~~Analyze the patterns for radiance data for highly damaged areas vs undamaged or low damage areas~~
* ~~Write up conclusions~~

### Query what images are relevant for what dates and what areas
* Fetch monthly catalogs of VIIRS NDB night lights imagery (2012-2020) 
* Create a searchable database of images for each day of all the night lights images
* Create an API to search and filter images by date and area boundary (as well as cache! the metadata)
* Create an API to evaluate overlap of images to area boundary
* Create an API to download selected images
  